In [28]:
import requests
import base64
from PIL import Image
from io import BytesIO
import json
import asyncio

import aiohttp

In [17]:
ip_address_llava = "3.68.222.21" #Have to be updated after start of EC2 instance
ip_address_medvet = "127.0.0.1:8000"

url = f"http://{ip_address_medvet}/generate"  
clear_url_llava = f"http://{ip_address_llava}/clearchat" 
clear_url_llama = f"http://{ip_address_medvet}/clearchat" 

In [33]:
def getImageBase64(image_name):
    if(image_name == "-" or image_name == ""):
        return ""
    img = Image.open(f"Images/{image_name}.png")
    rgb_im = img.convert('RGB')
    buffered = BytesIO()
    rgb_im.save(buffered, format="JPEG")  # You can change JPEG to PNG depending on your image format
    img_byte = buffered.getvalue()
    # Encode byte array to Base64
    img_base64 = base64.b64encode(img_byte).decode('utf-8')
    return img_base64

In [19]:
def clearChat():
    try:            
        if ip_address_llava != "":
            requests.post(clear_url_llava)
        if ip_address_medvet != "":
            requests.post(clear_url_llama)
    except Exception as e:
        print( f"Failed to connect to AWS:{e}")

In [46]:
def getAnswerMedVet(case,image_name,combined:bool):
    output = ""
    data = {
                "prompt": case,
                "agent_id": "validation_01",
                "ip_address_llava": ip_address_llava,
                "img": getImageBase64(image_name),
                "display_combined": combined,
                "mode_assistant": "Casesolver",
                "use_rag": 1,
                "temperature": 0.7,
                "max_new_tokens": 1024
            }
    headers = {'Content-type': 'application/json', 'Accept': 'application/json'}        
    try:
        response = requests.put(url, data=json.dumps(data), headers=headers)
        response = response.json()
        output = response
        
    except Exception as e:
        output = f"Failed to connect to AWS:{e}"
    return output

<h1>Combined Answers</h1>

In [38]:
import csv

lines_newCSV = []
with open('Evaluation Cases.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        answer = getAnswerMedVet(row['Case'],row['Image'],1)
        print(answer)
        lines_newCSV.append({
            "Number":line_count,
            "Case": row['Case'],
            "Image": row['Image'],
            "Solution": row['Solution'],
            "Source": row['Source'],
            "Answer MedVet": answer['response'],
            "Answer ChatGPT": row['Answer ChatGPT'],
            "Similarity MedVet - Solution": row['Similarity MedVet - Solution'],
            "Similarity ChatGPT - Solution": row['Similarity ChatGPT - Solution'],
            "Evaluation from Veterinarian": row['Evaluation from Veterinarian']
            })
        clearChat()
        line_count +=1
        

**Result LlaMA and RAG**

  Ah, I see. Based on the information provided, it's possible that Jake may have developed a dental condition known as pyorrhea. Pyorrhea is a common condition in dogs, especially those with heavy teeth grinding or chewing habits. It occurs when the gums become inflamed and bleed easily, which can cause bad breath and traces of blood on chew toys.

As Jake's primary care veterinarian, it's important to evaluate the extent of the pyorrhea and determine the underlying cause. A thorough dental examination, including radiographs (x-rays), may be necessary to identify any underlying dental problems, such as abscesses or calculus (tartar buildup) on the teeth.

Treatment for pyorrhea typically involves a combination of professional dental cleaning and management of any underlying conditions that may be contributing to the disease. In some cases, medication may be prescribed to help control inflammation and promote healing.

In addition to addressing the dental issue

In [41]:
with open('Results/MedVetAnswers_combined.csv', 'w', newline='') as csvfile:
    fieldnames = ['Number', "Case", 'Image','Solution','Source',"Answer MedVet","Answer ChatGPT","Similarity MedVet - Solution","Similarity ChatGPT - Solution","Evaluation from Veterinarian"] 
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(lines_newCSV)

<h1>Separate</h1>

In [47]:
import csv

lines_newCSV = []
with open('Evaluation Cases.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        answer = getAnswerMedVet(row['Case'],row['Image'],0)
        answer_llama = ""
        answer_llava = ""
        if "answer_llama" in answer:
            answer_llama = answer['answer_llama']['result']
        if "answer_llava" in answer:
            answer_llava = answer['answer_llava']['result']
            
        lines_newCSV.append({
            "Number":line_count,
            "Case": row['Case'],
            "Image": row['Image'],
            "Solution": row['Solution'],
            "Source": row['Source'],
            "Answer MedVet LlaMA": answer_llama,
            "Answer MedVet LlaVA": answer_llava,
            "Answer ChatGPT": row['Answer ChatGPT'],
            "Similarity MedVet - Solution": row['Similarity MedVet - Solution'],
            "Similarity ChatGPT - Solution": row['Similarity ChatGPT - Solution'],
            "Evaluation from Veterinarian": row['Evaluation from Veterinarian']
            })
        clearChat()
        line_count +=1
        

In [48]:
with open('Results/MedVetAnswers_separated.csv', 'w', newline='') as csvfile:
    fieldnames = ['Number', "Case", 'Image','Solution','Source',"Answer MedVet LlaMA","Answer MedVet LlaVA","Answer ChatGPT","Similarity MedVet - Solution","Similarity ChatGPT - Solution","Evaluation from Veterinarian"] 
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(lines_newCSV)